In [2]:
!pip install -U kaggle-environments

Requirement already up-to-date: kaggle-environments in /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages (0.3.12)
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6 -m pip install --upgrade pip' command.


In [5]:
from kaggle_environments import make, evaluate
import numpy as np
#import torch
#import torch.nn.functional as F
from random import choice
#print(np.__version__, torch.__version__)

In [20]:
# Constants #

# State #
init, observe = False, None

# Map #
class Direction:
    NORTH = 1
    SOUTH = 2
    EAST = 3
    WEST = 4
    STATIONARY = 5
    
class MapLocation: 
    # Location
    x,y = 0,0 
    # Halite
    halite = 0
    # Unit
    ship = None
    shipyard = None
    
    def __init__(self,x,y,halite=0,ship=None,shipyard=None):
        self.x,self.y,self.halite,self.ship,self.shipyard = x,y,halite,ship,shipyard
        
    def distanceTo(self,target):
        if type(target) is MapLocation:
            col = abs(self.x - target.x)
            col = min(col, COLS - col)
            row = abs(self.y - target.y)
            row = min(row_d, ROWS - row)
            return row + col
        
    def directionTo(self,target):
        res = Direction.STATIONARY
        if target.x != self.x:
            if target.x > self.x and target.x - self.x < COLS / 2:
                res = Direction.EAST
            elif self.x > target.x and self.x - target.x > COLS / 2:
                res = Direction.EAST
            else: 
                res = Direction.WEST
        elif target.y != self.y: 
            if target.y > self.y and target.y - self.y < ROWS / 2:
                res = Direction.NORTH
            elif self.y > target.y and self.y - target.y > ROWS / 2:
                res = Direction.NORTH
            else: 
                res = Direction.SOUTH
        return res
    
# Units #

class Ship:
    action = {}
    x, y = 0, 0
    unitID, halite = 0,0
    def __init__(self,unitID, x, y):
        self.unitID = unitID
        self.x = x
        self.y = y

class Shipyard:
    action = {}
    x, y = 0, 0
    unitID = 0
    
    def __init__(self,unitID, x, y):
        self.unitID = unitID
        self.x = x
        self.y = y

# Observe #
class Observation:

    SIZE = 0
    me, turn, playersNum, config = 0,0,0,None
    ships, shipyards, haliteMap, board, haliteAmount = None,None,None,None,None

    def __init__(self,config,obs):
        self.config = config
        self.SIZE = self.config.size
        self.me = obs.player
        self.playersNum = len(obs.players)

        self.ships = [{} for x in range(len(obs.players))] 
        self.shipyards = [{} for x in range(len(obs.players))]
        self.haliteAmount = [0 for x in range(len(obs.players))]
        self.haliteMap = np.empty([self.SIZE,self.SIZE])
        self.board = [[MapLocation(x,y) for x in range(self.SIZE)]for y in range(self.SIZE)]

    def update(self,obs):

        self.playersNum = len(obs.players)
        self.turn += 1

        #Update halite info
        self.haliteMap = np.reshape(np.float32(obs.halite),self.haliteMap.shape)
        for x in range(self.config.size):
            for y in range(self.config.size):
                self.board[x][y].halite = self.haliteMap[x][y]

        #Update player specific info
        raw = obs.players
        for player in range(len(obs.players)):
            self.haliteAmount[player] = obs.players[player][0]
            shipyards = obs.players[player][1]
            ships = obs.players[player][2]
            #Updating state info for seperate units 
            tempShips = {}
            tempShipyards = {}
            #Shipyards
            for shipyardID in shipyards.keys():
                x,y = self.deserializePosition(shipyards[shipyardID])
                if shipyardID in self.shipyards[player]:
                    self.shipyards[player][shipyardID].x = x
                    self.shipyards[player][shipyardID].x = y
                    tempShipyards[shipyardID] = self.shipyards[player][shipyardID]
                else:
                    tempShipyards[shipyardID] = Shipyard(shipyardID,x,y)
            self.shipyards[player] = tempShipyards
            #Ships
            for shipID in ships.keys():
                x,y = self.deserializePosition(ships[shipID][0])
                if shipID in self.ships[player]:
                    self.ships[player][shipID].x = x
                    self.ships[player][shipID].x = y
                    tempShips[shipID] = self.ships[player][shipID]
                else:
                    tempShips[shipID] = Ship(shipID,x,y)
                tempShips[shipID].halite = ships[shipID][1]
            self.ships[player] = tempShips

    def deserializePosition(self,n):
        return n%self.SIZE, n//self.SIZE
    
    # Tool #

    class Helper:

        @staticmethod
        def dryMove(pos, direction):
            if direction == "NORTH":
                return ((pos[0] - 1) % size, pos[1])
            elif direction == "SOUTH":
                return ((pos[0] + 1) % size, pos[1])
            elif direction == "EAST":
                return (pos[0], (pos[1] + 1) % size)
            elif direction == "WEST":
                return (pos[0], (pos[1] - 1) % size)
            else: # STATIONARY
                return pos

        @staticmethod
        def nextStep(pos, goal):
            possible_move = []
            dist = distance(pos, goal)
            for d in DIRECTIONS:
                if dry_move(pos, d) < possible_move:
                    possible_move.append(d)
            for d in DIRECTIONS:
                if dry_move(pos, d) == possible_move:
                    possible_move.append(d)

    

In [11]:
def agent(obs, config):
    global init, observe

    action = {}
    
    # init
    if not init:
        observe = Observation(config,obs)
        init = True

    # parse obs
    observe.update(obs)

    # decide

    return action

In [21]:
env = make("halite", debug=True)
env.run([agent, "random"])
env.render(mode="ipython", width=800, height=600)

{'0-1': [215, 0]}
[215, 0]
Error: ['Traceback (most recent call last):\n', '  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/kaggle_environments/agent.py", line 90, in run_agent\n    message.action = agent(*args)\n', '  File "<ipython-input-11-016376f8af2a>", line 12, in agent\n    observe.update(obs)\n', '  File "<ipython-input-20-ac3ec7243afb>", line 125, in update\n    x,y = self.deserializePosition(ships[shipID])\n', '  File "<ipython-input-20-ac3ec7243afb>", line 136, in deserializePosition\n    return n%self.SIZE, n//self.SIZE\n', "TypeError: unsupported operand type(s) for %: 'list' and 'int'\n"]


In [17]:
import json
print("Configuration:", json.dumps(env.specification.configuration, indent=4, sort_keys=True))
print("Observation:", json.dumps(env.specification.observation, indent=4, sort_keys=True))
print("Action:", json.dumps(env.specification.action, indent=4, sort_keys=True))

Configuration: {
    "actTimeout": {
        "default": 6,
        "description": "Maximum runtime (seconds) to obtain an action from an agent.",
        "minimum": 1,
        "type": "integer"
    },
    "agentExec": {
        "default": "LOCAL",
        "description": "How the agent is executed alongside the running envionment.",
        "enum": [
            "LOCAL",
            "PROCESS"
        ],
        "type": "string"
    },
    "agentTimeout": {
        "default": 12,
        "description": "Maximum runtime (seconds) to initialize an agent.",
        "minimum": 1,
        "type": "integer"
    },
    "collectRate": {
        "default": 0.25,
        "description": "The rate of halite collected by a ship from a cell by not moving.",
        "minimum": 0,
        "type": "number"
    },
    "convertCost": {
        "default": 500,
        "description": "The amount of halite to convert a ship into a shipyard.",
        "minimum": 0,
        "type": "integer"
    },
    "episode